## 1. Install dependencies

In [ ]:
import os
import locale
locale.getpreferredencoding = lambda: "UTF-8"

!nvidia-smi

# Setup paths
%cd /workspace
home = os.getcwd()
dataset_path = "/workspace/dataset"

%pip install -q ultralytics gdown

In [ ]:
!gdown --fuzzy 'https://drive.google.com/file/d/1dKFfXfO0opSOiH_Ocfcpiy3SJifzeJdy/view?usp=sharing' -O {home}/dataset.zip
!mkdir -p {home}/dataset
!rm -rf {home}/dataset
!unzip -q {home}/dataset.zip -d {home}/dataset

## 2. Setup Ultralytics

In [ ]:
import ultralytics
from ultralytics import YOLO

ultralytics.checks()

In [ ]:
dataset_path = f'/{home}/dataset'
runs_path = f'{home}/runs/detect'

## 3. Fine-tune YOLOv12 model on a Custom Dataset

In [ ]:
!rm -rf {runs_path}

In [ ]:
# choose one:

model_names = (
  "yolo12n.pt",
  # "yolo12s.pt",
  # "yolo12m.pt",
  # "yolo12l.pt",
  # "yolo12x.pt"
)

for nr, model_name in enumerate(model_names):
  train_nr = str(nr+1)
  if train_nr == "1":
    train_nr = ''
  if nr < -1: continue
  model = YOLO(model_name)
  results = model.train(
    data=f'{dataset_path}/data.yaml',
    batch=64, # batch size
    cos_lr=True, # adjusts the learning rate following cosine decay
    # use default learning rate (0.005)
    epochs=100, # train for at most N epochs
    patience=20, # stops if there's N epochs with no improvments
    # use default weight decay (0.005)
    # use default warmup epochs (3) and LR bias (0)

    deterministic=False, # small performance boost
    optimizer="AdamW",

    # NOTE: see online visualizations of the augmentations too
    mosaic=0.7, # mosaic augmentation chance
    close_mosaic=20, # stops mosaic augmentation for last N epochs
    copy_paste=0.2, # copy paste augmentation chance
    erasing=0.45, # erasing augmentation chance

    hsv_h=0.015, hsv_s=0.7, hsv_v=0.5, # max hue, saturation, value shift range
    scale=0.6, # max scaling range
    translate=0.2, # max translation range
    degrees=15, # max rotation range

    device=[0,1] # use multiple GPUs
  )

  best_model_path = f'{runs_path}/train{train_nr}/weights/best.pt'
  best_model = YOLO(best_model_path)

  results = best_model.val()
  results = best_model.val(split='test')
  results = best_model.predict(source = f"{dataset_path}/test/images", save = True)